### Manipulation of PeaskDB de novo-assisted database search results of Trocas 7 (high water, April 2019) incubation samples LC-MS/MS data using python.

Starting with:

    PeaksDB search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Exported at <1.0% FDR
    
Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the TROCAS # (e.g., 101) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

In [1]:
cd /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/406_TROCAS7_Fusion_Apr2021_PEAKS_178/

/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/406_TROCAS7_Fusion_Apr2021_PEAKS_178


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
ls 

 406_TROCAS7_Fusion_Apr2021_PEAKS_178_peptide.csv
 406_TROCAS7_Fusion_Apr2021_PEAKS_178_protein.csv
 406_TROCAS7_Fusion_Apr2021_PEAKS_178_protein.fasta
'DB search psm.csv'
'de novo only peptides.csv'
 peptides.pep.xml
 protein-peptides.csv


In [4]:
# read the CSV into a dataframe using the pandas read_csv function
peaksdbdup406 = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/T7-INC-UWPR_Apr2021_Fus_T00T24/DN50_10lgP15/406_TROCAS7_Fusion_Apr2021_PEAKS_178/406_TROCAS7_Fusion_Apr2021_PEAKS_178_peptide.csv")

# remove redundant rows
peaksdb406 = pd.DataFrame.drop_duplicates(peaksdbdup406)

print(peaksdb406.columns)

columns = ['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area', 'Fraction', 'Scan', 'Source_File',
       '#Spec', '#Spec', 'Accession', 'PTM',
       'AScore']

peaksdb406.columns = columns

#remmove # spec and accession columns because they mess parsing up

del peaksdb406['#Spec']
del peaksdb406['Accession']
del peaksdb406['PTM']
del peaksdb406['AScore']

mean_length = peaksdb406['Length'].mean()
print(mean_length)

print("# redundant peaksdb peptides in combined dataframe", len(peaksdbdup406))
print("# nonredundant peaksdb peptides in combined dataframe", len(peaksdb406))

#look at the dataframe
peaksdb406.head()

Index(['Peptide', '-10lgP', 'Mass', 'Length', 'ppm', 'm/z', 'RT',
       'Area Trocas7-406-Chav-DUP', 'Fraction', 'Scan', 'Source File', '#Spec',
       '#Spec Trocas7-406-Chav-DUP', 'Accession', 'PTM', 'AScore'],
      dtype='object')
10.220588235294118
# redundant peaksdb peptides in combined dataframe 136
# nonredundant peaksdb peptides in combined dataframe 136


,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,Source_File
0,SC(+57.02)AAAGTEC(+57.02)LISGWGNTK,75.04,1881.8350,18,3.2,941.9277,93.77,1470000.0,109,18239,20210411_Trocas7_655_Chav406_DDA_120min_2.raw
1,SSGSSYPSLLQC(+57.02)LK,61.06,1525.7446,14,2.6,763.8816,102.30,208000.0,109,19848,20210411_Trocas7_655_Chav406_DDA_120min_2.raw
2,LGEHNIDVLEGNEQFINAAK,59.11,2210.0967,20,1.1,737.7070,97.81,84900.0,109,19045,20210411_Trocas7_655_Chav406_DDA_120min_2.raw
3,APVLSDSSC(+57.02)K,49.89,1062.5016,10,1.2,532.2587,37.92,155000.0,109,7417,20210411_Trocas7_655_Chav406_DDA_120min_2.raw
4,AVM(+15.99)DDFAAFVEK,45.97,1357.6223,12,0.3,679.8186,131.91,4070.0,109,24246,20210411_Trocas7_655_Chav406_DDA_120min_2.raw


In [5]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaksdb406['A'] = peaksdb406['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaksdb406['C'] = peaksdb406['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaksdb406['D'] = peaksdb406['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaksdb406['E'] = peaksdb406['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaksdb406['F'] = peaksdb406['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaksdb406['G'] = peaksdb406['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaksdb406['H'] = peaksdb406['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaksdb406 output, there will be no isoleucines (they're lumped in with leucines)
peaksdb406['I'] = peaksdb406['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaksdb406['K'] = peaksdb406['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaksdb406['L'] = peaksdb406['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaksdb406['M'] = peaksdb406['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaksdb406['N'] = peaksdb406['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaksdb406['P'] = peaksdb406['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaksdb406['Q'] = peaksdb406['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaksdb406['R'] = peaksdb406['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaksdb406['S'] = peaksdb406['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaksdb406['T'] = peaksdb406['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaksdb406['V'] = peaksdb406['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaksdb406['W'] = peaksdb406['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaksdb406['Y'] = peaksdb406['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaksdb406['c-carb'] = peaksdb406['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaksdb406['m-oxid'] = peaksdb406['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaksdb406['n-deam'] = peaksdb406['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a lamba function to enumerate the # of deamidated Q's in each peptide
peaksdb406['q-deam'] = peaksdb406['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# create a column with 'stripped' peptide sequences using strip
peaksdb406['stripped_peptide'] = peaksdb406['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaksdb406['stripped_length'] = peaksdb406['stripped_peptide'].apply(len)

peaksdb406['NAAF_num.'] = peaksdb406['Area'] / peaksdb406['stripped_length']

# total the number of modifications in sequence
peaksdb406['ptm-total'] = peaksdb406['c-carb'] + peaksdb406['m-oxid'] + peaksdb406['n-deam'] + peaksdb406['q-deam']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
peaksdb406['stripped_IL']= peaksdb406['stripped_peptide'].str.replace('I','L')

# write modified dataframe to new txt file
peaksdb406.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/406_CV_T24_GD_PDB/406B_CV_T24_GD_PDB.csv")

# check out the results
peaksdb406.head()

,Peptide,-10lgP,Mass,Length,ppm,m/z,RT,Area,Fraction,Scan,...,Y,c-carb,m-oxid,n-deam,q-deam,stripped_peptide,stripped_length,NAAF_num.,ptm-total,stripped_IL
0,SC(+57.02)AAAGTEC(+57.02)LISGWGNTK,75.04,1881.8350,18,3.2,941.9277,93.77,1470000.0,109,18239,...,0,2,0,0,0,SCLISGWGNTK,11,133636.363636,2,SCLLSGWGNTK
1,SSGSSYPSLLQC(+57.02)LK,61.06,1525.7446,14,2.6,763.8816,102.30,208000.0,109,19848,...,1,1,0,0,0,SSGSSYPSLLQCLK,14,14857.142857,1,SSGSSYPSLLQCLK
2,LGEHNIDVLEGNEQFINAAK,59.11,2210.0967,20,1.1,737.7070,97.81,84900.0,109,19045,...,0,0,0,0,0,LGEHNIDVLEGNEQFINAAK,20,4245.000000,0,LGEHNLDVLEGNEQFLNAAK
3,APVLSDSSC(+57.02)K,49.89,1062.5016,10,1.2,532.2587,37.92,155000.0,109,7417,...,0,1,0,0,0,APVLSDSSCK,10,15500.000000,1,APVLSDSSCK
4,AVM(+15.99)DDFAAFVEK,45.97,1357.6223,12,0.3,679.8186,131.91,4070.0,109,24246,...,0,0,1,0,0,AVMDDFAAFVEK,12,339.166667,1,AVMDDFAAFVEK


In [6]:
# keep only stripped peptide I/L and NAAF
db_406 = peaksdb406[['stripped_IL', 'Area', 'NAAF_num.']]

db_406.set_index('stripped_IL')

# write modified dataframe to new txt file
db_406.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/406B_CV_T24_GD_PDB_ILnaafs.csv")

db_406.head()

,stripped_IL,Area,NAAF_num.
0,SCLLSGWGNTK,1470000.0,133636.363636
1,SSGSSYPSLLQCLK,208000.0,14857.142857
2,LGEHNLDVLEGNEQFLNAAK,84900.0,4245.000000
3,APVLSDSSCK,155000.0,15500.000000
4,AVMDDFAAFVEK,4070.0,339.166667


### Exporting txt files of stripped peptides at confidence cutoffs:

In [7]:
# keep only peptide column peptides <1% FDR (this is what we exported)
pep406moddup = peaksdb406[["Peptide"]]

# keep only the stripped peptide column <1% FDR
# this is what we'll use for UniPept input, etc
pep406dup = peaksdb406[["stripped_peptide"]]

# deduplicate both of these lists
pep406mod = pep406moddup.drop_duplicates()
pep406 = pep406dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified PeaksDB peptides in 406:", len(pep406moddup))
print("Deduplicated modified PeaksDB peptides in 406:", len(pep406mod))
print("Total stripped PeaksDB peptides in 406:", len(pep406dup))
print("Deduplicated stripped PeaksDB peptides in 406:", len(pep406))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep406.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/406_CV_T24_GD_PDB/406B_CV_T24_GD_PDB_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/406_CV_T24_GD_PDB/406B_CV_T24_GD_PDB_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/406_CV_T24_GD_PDB/406B_CV_T24_GD_PDB_stripped_peptides.fas

# look at the stripped peptides
pep406.head()

Total modified PeaksDB peptides in 406: 136
Deduplicated modified PeaksDB peptides in 406: 136
Total stripped PeaksDB peptides in 406: 136
Deduplicated stripped PeaksDB peptides in 406: 131


,stripped_peptide
0,SCLISGWGNTK
1,SSGSSYPSLLQCLK
2,LGEHNIDVLEGNEQFINAAK
3,APVLSDSSCK
4,AVMDDFAAFVEK


## NAAF correction and exporting files with AA and PTM totals:

In [8]:
# made a new dataframe that contains the suMN of certain columns in the modified
# peptide dataframe above 

index = ['sample total']

data = {'A': peaksdb406['A'].sum(),
        'C': peaksdb406['C'].sum(),
        'D': peaksdb406['D'].sum(),
        'E': peaksdb406['E'].sum(),
        'F': peaksdb406['F'].sum(),
        'G': peaksdb406['G'].sum(),
        'H': peaksdb406['H'].sum(),
        'I': peaksdb406['I'].sum(),
        'K': peaksdb406['K'].sum(),
        'L': peaksdb406['L'].sum(),
        'M': peaksdb406['M'].sum(),
        'N': peaksdb406['N'].sum(),
        'P': peaksdb406['P'].sum(),
        'Q': peaksdb406['Q'].sum(),
        'R': peaksdb406['R'].sum(),
        'S': peaksdb406['S'].sum(),
        'T': peaksdb406['T'].sum(),
        'V': peaksdb406['V'].sum(),
        'W': peaksdb406['W'].sum(),
        'Y': peaksdb406['Y'].sum(),
        'c-carb': peaksdb406['c-carb'].sum(),
        'm-oxid': peaksdb406['m-oxid'].sum(),
        'n-deam': peaksdb406['n-deam'].sum(),
        'q-deam': peaksdb406['q-deam'].sum(),
        'Total area': peaksdb406['Area'].sum(),
        'Total length': peaksdb406['stripped_length'].sum()
       }

totalpeaksdb406 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                              'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                              'c-carb', 'm-oxid', 
                                              'n-deam', 'q-deam', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaksdb406['% C w/ carb'] = totalpeaksdb406['c-carb'] / totalpeaksdb406['C'] 

# calculate percentage of M's that are oxidized
totalpeaksdb406['% M w/ oxid'] = totalpeaksdb406['m-oxid'] / totalpeaksdb406['M'] 

# calculate percentage of N's that are deamidated
totalpeaksdb406['% N w/ deam'] = totalpeaksdb406['n-deam'] / totalpeaksdb406['N'] 

# calculate percentage of N's that are deamidated
totalpeaksdb406['% Q w/ deam'] = totalpeaksdb406['q-deam'] / totalpeaksdb406['Q'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaksdb406['NAAF denom.'] = totalpeaksdb406['Total area'] / totalpeaksdb406['Total length']

# write modified dataframe to new txt file
totalpeaksdb406.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/406_CV_T24_GD_PDB/406B_CV_T24_GD_PDB_totals.csv")

totalpeaksdb406.head()

,A,C,D,E,F,G,H,I,K,L,...,m-oxid,n-deam,q-deam,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF denom.
sample total,134,23,64,83,47,112,15,89,67,144,...,10,22,24,244873890.0,1293,1.0,0.526316,0.247191,0.470588,189384.292343


In [9]:
# use the calculated NAAF factor (in totalpeaksdb406 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF20 = 189384.292343

# we'll use -10lgP > 20 as an approximate cutoff for the <1.0% FDR which is what we actually have
peaksdb406['NAAF factor'] = (peaksdb406['NAAF_num.'])/NAAF20

# make a dataframe that contains only what we need: sequences, AAs, PTMN
peaksdb406_NAAF = peaksdb406[['stripped_peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', \
                              'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                              'c-carb', 'm-oxid', 'n-deam', 'q-deam']].copy()

# multiply the NAAF20 factor by the AAs to normalize its abundance by peak area and peptide length

peaksdb406_NAAF['A-NAAF20'] = peaksdb406_NAAF['A'] * peaksdb406['NAAF factor']
peaksdb406_NAAF['C-NAAF20'] = peaksdb406_NAAF['C'] * peaksdb406['NAAF factor']
peaksdb406_NAAF['D-NAAF20'] = peaksdb406_NAAF['D'] * peaksdb406['NAAF factor']
peaksdb406_NAAF['E-NAAF20'] = peaksdb406_NAAF['E'] * peaksdb406['NAAF factor']
peaksdb406_NAAF['F-NAAF20'] = peaksdb406_NAAF['F'] * peaksdb406['NAAF factor']
peaksdb406_NAAF['G-NAAF20'] = peaksdb406_NAAF['G'] * peaksdb406['NAAF factor']
peaksdb406_NAAF['H-NAAF20'] = peaksdb406_NAAF['H'] * peaksdb406['NAAF factor']
peaksdb406_NAAF['I-NAAF20'] = peaksdb406_NAAF['I'] * peaksdb406['NAAF factor']
peaksdb406_NAAF['K-NAAF20'] = peaksdb406_NAAF['K'] * peaksdb406['NAAF factor']
peaksdb406_NAAF['L-NAAF20'] = peaksdb406_NAAF['L'] * peaksdb406['NAAF factor']
peaksdb406_NAAF['M-NAAF20'] = peaksdb406_NAAF['M'] * peaksdb406['NAAF factor']
peaksdb406_NAAF['N-NAAF20'] = peaksdb406_NAAF['N'] * peaksdb406['NAAF factor']
peaksdb406_NAAF['P-NAAF20'] = peaksdb406_NAAF['P'] * peaksdb406['NAAF factor']
peaksdb406_NAAF['Q-NAAF20'] = peaksdb406_NAAF['Q'] * peaksdb406['NAAF factor']
peaksdb406_NAAF['R-NAAF20'] = peaksdb406_NAAF['R'] * peaksdb406['NAAF factor']
peaksdb406_NAAF['S-NAAF20'] = peaksdb406_NAAF['S'] * peaksdb406['NAAF factor']
peaksdb406_NAAF['T-NAAF20'] = peaksdb406_NAAF['T'] * peaksdb406['NAAF factor']
peaksdb406_NAAF['V-NAAF20'] = peaksdb406_NAAF['V'] * peaksdb406['NAAF factor']
peaksdb406_NAAF['W-NAAF20'] = peaksdb406_NAAF['W'] * peaksdb406['NAAF factor']
peaksdb406_NAAF['Y-NAAF20'] = peaksdb406_NAAF['Y'] * peaksdb406['NAAF factor']

# multiply the NAAF20 factor by the PTMN normalize its abundance by peak area and peptide length

peaksdb406_NAAF['ccarb-NAAF20'] = peaksdb406_NAAF['c-carb'] * peaksdb406_NAAF['NAAF factor']
peaksdb406_NAAF['moxid-NAAF20'] = peaksdb406_NAAF['m-oxid'] * peaksdb406_NAAF['NAAF factor']
peaksdb406_NAAF['ndeam-NAAF20'] = peaksdb406_NAAF['n-deam'] * peaksdb406_NAAF['NAAF factor']
peaksdb406_NAAF['qdeam-NAAF20'] = peaksdb406_NAAF['q-deam'] * peaksdb406_NAAF['NAAF factor']


# write the dataframe to a new csv
peaksdb406_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/406_CV_T24_GD_PDB/406B_CV_T24_GD_PDB_naaf.csv")

peaksdb406_NAAF.head()

,stripped_peptide,NAAF factor,A,C,D,E,F,G,H,I,...,R-NAAF20,S-NAAF20,T-NAAF20,V-NAAF20,W-NAAF20,Y-NAAF20,ccarb-NAAF20,moxid-NAAF20,ndeam-NAAF20,qdeam-NAAF20
0,SCLISGWGNTK,0.705636,3,2,0,1,0,3,0,1,...,0.0,1.411272,1.411272,0.000000,0.705636,0.00000,1.411272,0.000000,0.0,0.0
1,SSGSSYPSLLQCLK,0.078450,0,1,0,0,0,1,0,0,...,0.0,0.392249,0.000000,0.000000,0.000000,0.07845,0.078450,0.000000,0.0,0.0
2,LGEHNIDVLEGNEQFINAAK,0.022415,2,0,1,3,1,2,1,2,...,0.0,0.000000,0.000000,0.022415,0.000000,0.00000,0.000000,0.000000,0.0,0.0
3,APVLSDSSCK,0.081844,1,1,1,0,0,0,0,0,...,0.0,0.245533,0.000000,0.081844,0.000000,0.00000,0.081844,0.000000,0.0,0.0
4,AVMDDFAAFVEK,0.001791,3,0,2,1,2,0,0,0,...,0.0,0.000000,0.000000,0.003582,0.000000,0.00000,0.000000,0.001791,0.0,0.0


In [10]:
# made a new dataframe that contains the suMN of NAAF normalized AAs for peaksdb406 peaksdb results
# also contains the suMN of the NAAF-corrected PTMN occurances for each affected residue

index = ['sample total']

data = {'NAAF': peaksdb406_NAAF['NAAF factor'].sum(),
        'A': peaksdb406_NAAF['A-NAAF20'].sum(),
        'C': peaksdb406_NAAF['C-NAAF20'].sum(),
        'D': peaksdb406_NAAF['D-NAAF20'].sum(),
        'E': peaksdb406_NAAF['E-NAAF20'].sum(),
        'F': peaksdb406_NAAF['F-NAAF20'].sum(),
        'G': peaksdb406_NAAF['G-NAAF20'].sum(),
        'H': peaksdb406_NAAF['H-NAAF20'].sum(),
        'I': peaksdb406_NAAF['I-NAAF20'].sum(),
        'K': peaksdb406_NAAF['K-NAAF20'].sum(),
        'L': peaksdb406_NAAF['L-NAAF20'].sum(),
        'M': peaksdb406_NAAF['M-NAAF20'].sum(),
        'N': peaksdb406_NAAF['N-NAAF20'].sum(),
        'P': peaksdb406_NAAF['P-NAAF20'].sum(),
        'Q': peaksdb406_NAAF['Q-NAAF20'].sum(),
        'R': peaksdb406_NAAF['R-NAAF20'].sum(),
        'S': peaksdb406_NAAF['S-NAAF20'].sum(),
        'T': peaksdb406_NAAF['T-NAAF20'].sum(),
        'V': peaksdb406_NAAF['V-NAAF20'].sum(),
        'W': peaksdb406_NAAF['W-NAAF20'].sum(),
        'Y': peaksdb406_NAAF['Y-NAAF20'].sum(),
        'c-carb': peaksdb406_NAAF['ccarb-NAAF20'].sum(),
        'm-oxid': peaksdb406_NAAF['moxid-NAAF20'].sum(),
        'n-deam': peaksdb406_NAAF['ndeam-NAAF20'].sum(),
        'q-deam': peaksdb406_NAAF['qdeam-NAAF20'].sum()
       }

totalpeaksdb406_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', \
                                                   'G', 'H', 'I','K', 'L', 'M', \
                                                   'N', 'P', 'Q', 'R', 'S', \
                                                   'T', 'V', 'W', 'Y', 'c-carb', \
                                                   'm-oxid', 'n-deam', 'q-deam'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaksdb406_NAAF['% C w/ carb.'] = totalpeaksdb406_NAAF['c-carb'] / totalpeaksdb406_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaksdb406_NAAF['% M w/ oxid'] = totalpeaksdb406_NAAF['m-oxid'] / totalpeaksdb406_NAAF['M'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb406_NAAF['% N w/ deam'] = totalpeaksdb406_NAAF['n-deam'] / totalpeaksdb406_NAAF['N'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaksdb406_NAAF['% Q w/ deam'] = totalpeaksdb406_NAAF['q-deam'] / totalpeaksdb406_NAAF['Q'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaksdb406_NAAF['NAAF check'] = totalpeaksdb406_NAAF['NAAF'] / 189384.292343

# write modified dataframe to new txt file, same name + totals
totalpeaksdb406_NAAF.to_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/PeaksDB/406_CV_T24_GD_PDB/406B_CV_T24_GD_PDB_naaf_totals.csv")

totalpeaksdb406_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,Y,c-carb,m-oxid,n-deam,q-deam,% C w/ carb.,% M w/ oxid,% N w/ deam,% Q w/ deam,NAAF check
sample total,154.494226,122.966464,3.105851,5.983102,5.962708,2.983438,9.399941,1.921631,20.402398,4.973162,...,2.22857,3.105851,0.510044,1.292912,1.119892,1.0,0.521841,0.041984,0.431873,0.000816
